## Note: Only run the executed cells

In [3]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
user_secrets = UserSecretsClient()
hf = user_secrets.get_secret("HF")
login(hf)

In [1]:
! pip show hf_transfer

Name: hf_transfer
Version: 0.1.9
Summary: Speed up file transfers with the Hugging Face Hub.
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [2]:
%env HF_ENABLE_HF_TRANSFER=1

env: HF_ENABLE_HF_TRANSFER=1


In [7]:
from transformers import AutoModelForCausalLM
import torch

model_id = "HuggingFaceTB/SmolLM-139M-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id)
print(model)

OSError: HuggingFaceTB/SmolLM-139M-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM-135M-Instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

2025-07-15 17:38:04.530004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752601084.842536      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752601084.936692      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [10]:
up= "./upload"
model.save_pretrained("./upload")
tokenizer.save_pretrained("./upload")

('./upload/tokenizer_config.json',
 './upload/special_tokens_map.json',
 './upload/chat_template.jinja',
 './upload/vocab.json',
 './upload/merges.txt',
 './upload/added_tokens.json',
 './upload/tokenizer.json')

In [11]:
! mv ./upload ./original

## Width prunning ,Layer Prunning
### Only Layer Prunning
### Prune only every 2nd last laye r 
### Naive prunning
Layer + Width prunning -> Naive Prunning

### Layer Prunning

In [27]:
import torch
import math

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def prune_layers(model, target_params, original_params):
    total_layers = len(model.model.layers)

    # Calculate number of layers to keep
    layers_to_keep = round(target_params / original_params * total_layers)
    layers_to_remove = total_layers - layers_to_keep

    # Keep all layers except those right before the last layer
    # selected_layers = list(model.model.layers[:total_layers - layers_to_remove - 1]) + \
    #                   model.model.layers[-1:]
    # Ensure all layers are selected except the ones we want to remove
    selected_layers = list(model.model.layers[:total_layers - layers_to_remove - 1]) + \
                       list(model.model.layers[-1:])

    
    model.model.layers = torch.nn.ModuleList(selected_layers)

    # Update the config
    model.config.num_hidden_layers = len(selected_layers)

    return model



def prune_hidden_dimensions(model, target_params, current_params):
    original_hidden_size = model.config.hidden_size
    original_intermediate_size = model.config.intermediate_size
    original_proj_ratio = original_intermediate_size / original_hidden_size # Calculate the ratio dynamically
    num_heads = model.config.num_attention_heads

    # Estimate new hidden size to target parameters
    reduction_ratio = math.sqrt(target_params / current_params)
    new_hidden_size = int(original_hidden_size * reduction_ratio)
    new_hidden_size = (new_hidden_size // (2 * num_heads)) * (2 * num_heads) # Ensure divisibility

    num_attention_heads = model.config.num_attention_heads
    num_key_value_heads = model.config.num_key_value_heads

    # Update hidden size and intermediate size in the config
    model.config.hidden_size = new_hidden_size
    model.config.intermediate_size = int(new_hidden_size * original_proj_ratio) # Maintain the original ratio

    for layer in model.model.layers:
        # Adjust attention projection layers
        layer.self_attn.q_proj.weight = torch.nn.Parameter(
            torch.empty(new_hidden_size, layer.self_attn.q_proj.weight.shape[1], device=layer.self_attn.q_proj.weight.device, dtype=layer.self_attn.q_proj.weight.dtype).contiguous()
        )
        layer.self_attn.k_proj.weight = torch.nn.Parameter(
            torch.empty(new_hidden_size, layer.self_attn.k_proj.weight.shape[1], device=layer.self_attn.k_proj.weight.device, dtype=layer.self_attn.k_proj.weight.dtype).contiguous()
        )
        layer.self_attn.v_proj.weight = torch.nn.Parameter(
            torch.empty(new_hidden_size, layer.self_attn.v_proj.weight.shape[1], device=layer.self_attn.v_proj.weight.device, dtype=layer.self_attn.v_proj.weight.dtype).contiguous()
        )
        layer.self_attn.o_proj.weight = torch.nn.Parameter(
            torch.empty(layer.self_attn.o_proj.weight.shape[0], new_hidden_size, device=layer.self_attn.o_proj.weight.device, dtype=layer.self_attn.o_proj.weight.dtype).contiguous()
        )

        # Adjust MLP layers
        new_intermediate_size = model.config.intermediate_size
        layer.mlp.gate_proj.weight = torch.nn.Parameter(
            torch.empty(new_intermediate_size, new_hidden_size, device=layer.mlp.gate_proj.weight.device, dtype=layer.mlp.gate_proj.weight.dtype).contiguous()
        )
        layer.mlp.up_proj.weight = torch.nn.Parameter(
            torch.empty(new_intermediate_size, new_hidden_size, device=layer.mlp.up_proj.weight.device, dtype=layer.mlp.up_proj.weight.dtype).contiguous()
        )
        layer.mlp.down_proj.weight = torch.nn.Parameter(
            torch.empty(new_hidden_size, new_intermediate_size, device=layer.mlp.down_proj.weight.device, dtype=layer.mlp.down_proj.weight.dtype).contiguous()
        )

    # Adjust rotary positional embeddings
    rotary_dim = new_hidden_size // num_heads
    model.model.rotary_emb.inv_freq = model.model.rotary_emb.inv_freq[:rotary_dim] # Assuming `inv_freq` is a tensor and needs truncation

    return model

In [24]:
def create_lm(model_name, target_params_1, target_params_2):

    model = AutoModelForCausalLM.from_pretrained(model_name)
    print("Original model:")
    print(model)

    original_params = count_parameters(model)
    print(f"Original model parameters: {original_params:,}.")

    # Step 2: Prune layers to target ~200M parameters
    model = prune_layers(model, target_params_1, original_params)
    new_params = count_parameters(model)
    print(f"Model parameters after layer pruning: {new_params:,}.")

    # Step 3: Prune hidden dimensions to target ~100M parameters
    model = prune_hidden_dimensions(model, target_params_2, new_params)

    final_params = count_parameters(model)
    print(f"Model parameters after hidden dimension pruning: {final_params:,}.")

    reduction_percentage = (1 - final_params / original_params) * 100
    print(f"Total size reduction: {reduction_percentage:.2f}%")

    return model

In [21]:
a=model.parameters()
for _ in a:
    print(_)

Parameter containing:
tensor([[-0.3711, -0.2158,  0.0327,  ...,  0.2314,  0.0444,  0.0586],
        [-0.0410,  0.0352,  0.0175,  ..., -0.0588, -0.0058, -0.0267],
        [-0.0228,  0.0221,  0.0059,  ..., -0.0258, -0.0398, -0.0454],
        ...,
        [ 0.1689, -0.0684, -0.0933,  ..., -0.0527, -0.1875, -0.1631],
        [-0.0532,  0.0579,  0.0184,  ...,  0.0874, -0.1797,  0.1982],
        [ 0.2598,  0.1230,  0.2070,  ..., -0.0952,  0.2021, -0.3633]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.2734, -0.0452, -0.0159,  ..., -0.0579, -0.1572, -0.1226],
        [ 0.3203,  0.1006, -0.1152,  ..., -0.0298, -0.0732,  0.0173],
        [ 0.0427, -0.0010,  0.0645,  ...,  0.0967,  0.1226,  0.0145],
        ...,
        [ 0.0400, -0.0801,  0.0190,  ..., -0.0040,  0.0233, -0.0053],
        [-0.0488,  0.0825, -0.0182,  ...,  0.0031, -0.0214,  0.0061],
        [ 0.0889, -0.0918,  0.0127,  ...,  0.0042,  0.0135, -0.0091]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.

In [22]:
count_parameters(model)

134515008

In [28]:

# Model name and target parameters
# model_name = "HuggingFaceTB/SmolLM-360M-Instruct"
# target_params_1 = 200_000_000  # Target ~200M parameters after layer pruning
# target_params_2 = 100_000_000  # Target ~100M parameters after hidden dimension pruning

model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
target_params_1 = 130_000_000
target_params_2 = 90_000_000


pruned_lm = create_lm(model_name, target_params_1, target_params_2)

modified_model_path = f"model_{model_name.split('/')[-1]}-layer-pruned-{int(target_params_2/1000000)}M-raw"
pruned_lm.save_pretrained(modified_model_path)
print(f"TrellisLM-100M-Instruct saved to: {modified_model_path}")

Original model:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)


## Push to Hub

In [ ]:
# Load the model from the original
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(original_model_name)

# Push the model to the hub
# model.push_to_hub(repo_name)

# Push the tokenizer to the hub
# tokenizer.push_to_hub(repo_name)

# print(f"Model and tokenizer pushed successfully to {repo_name}")

# Path to your local model is "modified_model_path"
local_model_path = modified_model_path # No need to push the model you just pruned
local_model_path = "SmolLM-360M-Instruct-layer-pruned-200M-raw"

# Set your repository name
repository_name = f"Trellis/{local_model_path}"

# Original model name
original_model_name = "HuggingFaceTB/SmolLM-360M-Instruct"
original_model_name = "HuggingFaceTB/SmolLM-139M-Instruct"

# Push the model
# push_model_to_hub(local_model_path, original_model_name, repository_name)

In [31]:
count_parameters(pruned_lm)

110428992

In [32]:
up= "./new"
pruned_lm.save_pretrained(up)
# tokenizer.save_pretrained("./upload")

In [38]:
from huggingface_hub import HfApi
import os
user_secrets = UserSecretsClient()
hf = user_secrets.get_secret("HF")
# api = HfApi(token=os.getenv("HF"))
api = HfApi(token=hf)
api.upload_folder(
    folder_path="./",
    repo_id="Pingsz/pruned",
    repo_type="model",
)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Pingsz/pruned/commit/f2a9fdfe96dc2b3deef233005d4cb6302d45353f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f2a9fdfe96dc2b3deef233005d4cb6302d45353f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pingsz/pruned', endpoint='https://huggingface.co', repo_type='model', repo_id='Pingsz/pruned'), pr_revision=None, pr_num=None)

In [39]:
! du -sh .

1.4G	.
